# ede Neural Quântica (VQC) com PCA no Student Performance

Este notebook treina e avalia uma Rede Neural Quântica (VQC) no dataset Student Performance, utilizando de Redução de Dimensionalidade com PCA.

In [1]:

import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import COBYLA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


from imblearn.over_sampling import SMOTE

In [ ]:

path = '../data/student-por.csv'
df_completo = pd.read_csv(path, sep=',')


y_grades = df_completo[['G1', 'G2', 'G3']]
X_features = df_completo.drop(columns=['G1', 'G2', 'G3'])


y_final = np.where(y_grades['G3'] >= 10, 1, 0)


X_final = pd.get_dummies(X_features, drop_first=True)

print("--- Dataset Student Performance preparado ---")
print("Formato das features (X_final):", X_final.shape)

--- Dataset Student Performance preparado ---
Formato das features (X_final): (649, 39)


In [3]:

X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test_full)
print("Dados originais divididos e normalizados.")


N_COMPONENTES = 5 
pca = PCA(n_components=N_COMPONENTES)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"PCA aplicado. Dimensões reduzidas para {N_COMPONENTES} componentes.")

Dados originais divididos e normalizados.
PCA aplicado. Dimensões reduzidas para 5 componentes.


In [4]:

smote = SMOTE(random_state=42)
X_train_resampled_pca, y_train_resampled = smote.fit_resample(X_train_pca, y_train)

print("\n--- Balanceamento do Conjunto de Treino DEPOIS do SMOTE ---")
print(pd.Series(y_train_resampled).value_counts())


--- Balanceamento do Conjunto de Treino DEPOIS do SMOTE ---
1    411
0    411
Name: count, dtype: int64


In [ ]:


num_features = X_train_resampled_pca.shape[1]  

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
optimizer = COBYLA(maxiter=100)
sampler = StatevectorSampler()


num_parametros = ansatz.num_parameters
rng = np.random.default_rng(42)
initial_point = rng.random(num_parametros)


vqc_sp_pca = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point
)

print(f"\nInstância do VQC criada para {num_features} features")


start_time = time.time()
vqc_sp_pca.fit(X_train_resampled_pca, y_train_resampled)
end_time = time.time()
print(f"Modelo VQC treinado em {end_time - start_time:.2f} segundos.")

vqc_predictions_pca = vqc_sp_pca.predict(X_test_pca)

print("\nRelatório de Classificação para a QNN/VQC com PCA (Student Performance(Port))")
print(classification_report(y_test, vqc_predictions_pca, target_names=['Reprovado', 'Aprovado']))

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



Instância do VQC criada para 5 features. Iniciando treinamento...
Modelo VQC treinado em 933.07 segundos.

--- Relatório de Classificação para a QNN/VQC com PCA (Student Performance) ---
              precision    recall  f1-score   support

   Reprovado       0.15      0.60      0.24        25
    Aprovado       0.85      0.40      0.54       138

    accuracy                           0.43       163
   macro avg       0.50      0.50      0.39       163
weighted avg       0.74      0.43      0.50       163

